In [1]:
from y0.dsl import One, P, A, B, C, D, Q, R, S, T, W, X, Y, Z, Sum, Variable, Product

In [2]:
One()

One()

In [3]:
P(A)

P(A)

In [4]:
P(A @ W)

P(A @ W)

In [5]:
P(A @ ~W)

P(A @ ~W)

In [6]:
P(~A @ W)

P(A @ W)

In [7]:
P(~A @ ~W)

P(A @ ~W)

In [8]:
P(A @ ~W @ X)

P(A @ (~W, X))

In [9]:
~A

~A

In [10]:
P(A, B)

P(A, B)

In [11]:
P(A | B)

P(A | B)

In [12]:
One() / P(A | B)

((One() / P(A | B)))

In [13]:
(One() / P(A | B)) * P(A)

((P(A) / P(A | B)))

In [14]:
One() * P(A | B)

P(A | B)

In [15]:
P(A | B) * P(B)

P(A | B) * P(B)

In [16]:
Sum[B](P(A | B))

Sum[B](P(A | B))

In [17]:
Sum(P(A | B) * Sum(P(C | D), [R]), [S, T])

Sum[S, T](P(A | B) * Sum[R](P(C | D)))

In [18]:
Sum[W](P((Y @ ~Z @ W) & X) * P(D) * P(Z @ D) * P(W @ ~X))

Sum[W](P(X, Y @ (~Z, W)) * P(D) * P(Z @ D) * P(W @ ~X))

In [19]:
(P(A) / P(B)) / (P(C) / P(D))

((P(A) * P(D) / P(B) * P(C)))

In [20]:
Q[A](B)

Q[A](B)

In [21]:
P(Variable(v) for v in ['A', 'B', 'C'])

P(A, B, C)

In [22]:
Product([P(A), P(B), P(C)])

P(A) * P(B) * P(C)

In [23]:
Sum(P(Variable(v) for v in ['A','B','C']), [Variable(v) for v in ['A', 'B']])

Sum[A, B](P(A, B, C))